# Tests for calculating confidence interval

In [58]:
import scipy.stats as st
import math
import statsmodels as sm


#### Z-test confidence interval

upper/lower bound of a confidence interval:<br>
X ± Z*(s/√n)<br>
X is the mean<br>
Z is the value from the standard normal distribution for the selected confidence level<br>
s is the standard deviation<br>
n is the number of observations<br>

A Z-test is any statistical test for which the distribution of the test statistic under the null hypothesis can be approximated by a normal distribution. Z-tests test the mean of a distribution. For each significance level in the confidence interval, the Z-test has a single critical value (for example, 1.96 for 5% two tailed).

In [59]:
# Z-Test


def ztest(n, mean, std_dev, conf):
    '''
    This test assumes that data is normally distributed and works well for bigger number of samples (>30).
    Function takes number of samples (n), mean value (mean),
    standard deviation (std_dev) and confidence (conf).
    Returns lower and upper bounds for the confidence interval.
    '''
    z = st.norm.ppf(1-(1-conf)/2)
    upper_bound = mean + z*std_dev/math.sqrt(n)
    lower_bound = mean - z*std_dev/math.sqrt(n)
    
    return [lower_bound, upper_bound]

print(ztest(100, 50, 5, 0.95))


def reverse_ztest(mean, std_dev, diff, conf):
    '''
    Function takes mean of data (mean), standard deviation of data (std_dev),
    difference from mean to lower/upper bound which is upper_bound-mean or mean-lower_bound (diff)
    and confidence (conf).
    Returns rounded number of samples which should be taken to obtain a given confidence interval.
    '''
    z = st.norm.ppf(1-(1-conf)/2)
    n = (z*std_dev/(diff))**2
    return int(round(n))

print(reverse_ztest(50, 5, 1.39, 0.95))


[49.02001800772997, 50.97998199227003]
50


#### Z-test with precision

In [60]:
# Z-Test precission

def ztest_pr(n, mean, conf):
    '''
    This test assumes that data is normally distributed and works well for bigger number of samples (>30).
    Function takes number of samples (n), mean value (mean) and confidence (conf).
    Returns lower and upper bounds for the confidence interval.
    '''
    z = st.norm.ppf(1-(1-conf)/2)
    pr = z*math.sqrt(0.25/n)
    upper_bound = mean + 10*pr
    lower_bound = mean - 10*pr
    
    return [lower_bound, upper_bound]

print(ztest_pr(50, 50, 0.95))


def reverse_ztest_pr(mean, diff, conf):
    '''
    Function takes mean of data (mean), difference from mean to lower/upper bound which is upper_bound-mean 
    or mean-lower_bound (diff) and confidence (conf).
    Returns rounded number of samples which should be taken to obtain a given confidence interval.
    '''
    z = st.norm.ppf(1-(1-conf)/2)
    n = (z*math.sqrt(0.25)/(diff/10))**2
    return int(round(n))

print(reverse_ztest_pr(50, 1.39, 0.95))

[48.614096175650324, 51.385903824349676]
50


#### t-test confidence interval

upper/lower bound of a confidence interval:<br>
X ± t*(s/√n)<br>
X is the mean<br>
t is the value from the Student's t-distribution for the selected confidence level<br>
s is the standard deviation<br>
n is the number of observations<br>

The t-test is any statistical hypothesis test in which the test statistic follows a Student's t-distribution under the null hypothesis.
A t-test is the most commonly applied when the test statistic would follow a normal distribution if the value of a scaling term in the test statistic were known.

In [61]:
# T-test

def ttest(n, mean, std_dev, conf):
    '''
    This test works for smaller number of samples (<30), uses t-distribution instead the normal one.
    Function takes number of samples (n), mean value (mean), standard deviation and confidence (conf).
    Returns lower and upper bounds for the confidence interval.
    '''
    t = st.t.ppf(1-(1-conf)/2, n-1)
    upper_bound = mean + t*std_dev/math.sqrt(n)
    lower_bound = mean - t*std_dev/math.sqrt(n)
    
    return [lower_bound, upper_bound]

ttest(9, 50, 3, 0.95)


#def reverse_ttest()

[47.69399586496663, 52.30600413503337]

#### Loose test set bound (Langford)

In [62]:
# Loose test set bound Langford

def loose_langford(n, mean, conf):
    '''
    Function takes number of samples (n), mean value (mean) and confidence (conf).
    Returns lower and upper bounds for the confidence interval.
    '''
    pr = math.sqrt(math.log(2/(1-conf))/(n*2))
    upper_bound = mean + pr*10
    lower_bound = mean - pr*10
    return [lower_bound, upper_bound]

print(loose_langford(50, 50, 0.95))

def loose_langford_reverse(mean, diff, conf):
    '''
    Function takes mean of data (mean), difference from mean to lower/upper bound which is upper_bound-mean 
    or mean-lower_bound (diff) and confidence (conf).
    Returns rounded number of samples which should be taken to obtain a given confidence interval.
    '''
    n = math.log(2/(1-conf))/(2*(diff/10)**2)
    return int(round(n))

print(loose_langford_reverse(50, 1.92, 0.95))

[48.07935441736016, 51.92064558263984]
50


#### Clopper-Pearson (beta distribution)
The Clopper–Pearson interval is an early and very common method for calculating binomial confidence intervals. This is often called an 'exact' method, because it is based on the cumulative probabilities of the binomial distribution (i.e., exactly the correct distribution rather than an approximation). However, in cases where we know the population size, the intervals may not be the smallest possible.

In [63]:
# Clopper-Pearson

def clopper_pearson(n, mean, conf):
    '''
    Function takes number of samples (n), mean value (mean) and confidence (conf).
    Returns lower and upper bounds for the confidence interval.
    '''
    low, high = sm.stats.proportion.proportion_confint(n/2, n, alpha=1-conf, method = "beta")
    return [mean-(0.5-low)*10, mean+(high-0.5)*10]

print(clopper_pearson(50, 50, 0.95))

[48.55272997129908, 51.44727002870092]


#### Wilson score interval
The Wilson score interval is an improvement over the normal approximation interval in multiple respects. Unlike the symmetric normal approximation interval, the Wilson score interval is asymmetric. It does not suffer from problems of overshoot and zero-width intervals that afflict the normal interval, and it may be safely employed with small samples and skewed observations.<br>
The Wilson interval is derived from the Wilson Score Test, which belongs to a class of tests called Rao Score Tests. It relies on the asymptotic normality of your estimator.
Wilson intervals get their assymetry from the underlying likelihood function for the binomial, which is used to compute the "expected standard error" and "score" (i.e., first derivative of the likelihood function) under the null hypotheisis. Since these values will change as you very your null hypothesis, the interval where the normalized score (score/expected standard error) exceeds your pre-specified Z-cutoff for significance will not be symmetric, in general.

In [64]:
# Wilson score interval

def wilson(n, mean, conf):
    '''
    Function takes number of samples (n), mean value (mean) and confidence (conf).
    Returns lower and upper bounds for the confidence interval.
    '''
    low, high = sm.stats.proportion.proportion_confint(n/2, n, alpha=1-conf, method = "wilson")
    return [mean-(0.5-low)*10, mean+(high-0.5)*10]

print(wilson(50, 50, 0.95))

[48.664451431682856, 51.335548568317144]
